In [7]:
!pip install transformers torch pandas

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.nn import BCEWithLogitsLoss
import pandas as pd

In [9]:
class BertPreprocessor:
    def __init__(self, model_name='bert-base-uncased', max_length=256):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess(self, texts):
        encoding = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding['input_ids'], encoding['attention_mask']

In [10]:
class BertModel:
    def __init__(self, model_name='bert-base-uncased'):
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train(self, input_ids, attention_masks, labels, batch_size=32, epochs=4):
        dataset = TensorDataset(input_ids, attention_masks, labels)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        loss_fct = BCEWithLogitsLoss()

        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in train_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                self.model.zero_grad()

                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits

                b_labels = b_labels.unsqueeze(1)
                b_labels = torch.cat((1 - b_labels, b_labels), dim=1)

                loss = loss_fct(logits, b_labels.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.2f}")

            self.model.eval()
            total_eval_accuracy = 0
            for batch in val_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                with torch.no_grad():
                    outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                predictions = np.argmax(logits, axis=1)
                total_eval_accuracy += accuracy_score(label_ids, predictions)

            avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy:.2f}")

    def evaluate(self, input_ids, attention_masks, labels, batch_size=32):
        test_dataset = TensorDataset(input_ids, attention_masks, labels)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        total_eval_accuracy = 0
        predictions, true_labels = [], []

        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
            with torch.no_grad():
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions)
            true_labels.extend(label_ids)

        print("Accuracy:", accuracy_score(true_labels, predictions))
        print("Classification Report:")
        print(classification_report(true_labels, predictions))

In [11]:
from google.colab import files
uploaded = files.upload()

Saving train_data.tsv to train_data (1).tsv
Saving test_data.tsv to test_data (1).tsv
Saving Validation_data.tsv to Validation_data (1).tsv


In [12]:
def main():
    # Load datasets
    train_df = pd.read_csv('train_data.tsv', sep='\t')
    test_df = pd.read_csv('test_data.tsv', sep='\t')
    val_df = pd.read_csv('Validation_data.tsv', sep='\t')

    # Define column names and label mapping
    columns = ['index', 'id', 'label', 'statement', 'subject', 'speaker', 'JobTitle', 'State', 'Party', 'BTC', 'FC', 'HT', 'MT', 'POF', 'context', 'justification']
    train_df.columns = columns
    test_df.columns = columns
    val_df.columns = columns

    label_mapping = {
        'true': 1, 'half-true': 1, 'mostly-true': 1,
        'false': 0, 'pants-fire': 0, 'barely-true': 0
    }

    # Apply label mapping to each dataset
    train_df['label'] = train_df['label'].map(label_mapping)
    test_df['label'] = test_df['label'].map(label_mapping)
    val_df['label'] = val_df['label'].map(label_mapping)

    # Preprocess the text data for BERT model
    bert_preprocessor = BertPreprocessor()

    # Preprocess Train Data
    train_texts = train_df['statement'].tolist()
    train_input_ids, train_attention_masks = bert_preprocessor.preprocess(train_texts)
    train_labels = torch.tensor(train_df['label'].values)

    # Preprocess Validation Data
    val_texts = val_df['statement'].tolist()
    val_input_ids, val_attention_masks = bert_preprocessor.preprocess(val_texts)
    val_labels = torch.tensor(val_df['label'].values)

    # Preprocess Test Data
    test_texts = test_df['statement'].tolist()
    test_input_ids, test_attention_masks = bert_preprocessor.preprocess(test_texts)
    test_labels = torch.tensor(test_df['label'].values)

    # Train and evaluate the BERT model
    bert_model = BertModel()
    bert_model.train(train_input_ids, train_attention_masks, train_labels)
    print("Validation Results:")
    bert_model.evaluate(val_input_ids, val_attention_masks, val_labels)
    print("Test Results:")
    bert_model.evaluate(test_input_ids, test_attention_masks, test_labels)
    print("Train Results:")
    bert_model.evaluate(train_input_ids, train_attention_masks, train_labels)


main()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 - Loss: 0.66
Validation Accuracy: 0.64
Epoch 2/4 - Loss: 0.61
Validation Accuracy: 0.64
Epoch 3/4 - Loss: 0.49
Validation Accuracy: 0.64
Epoch 4/4 - Loss: 0.29
Validation Accuracy: 0.63
Validation Results:
Accuracy: 0.8144972720187061
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.78      0.80       615
           1       0.80      0.85      0.83       668

    accuracy                           0.81      1283
   macro avg       0.82      0.81      0.81      1283
weighted avg       0.82      0.81      0.81      1283

Test Results:
Accuracy: 0.8206951026856241
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.77      0.79       553
           1       0.83      0.86      0.84       713

    accuracy                           0.82      1266
   macro avg       0.82      0.82      0.82      1266
weighted avg       0.82      0.82      0.82      1266

Train Result